[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/y-akbal/Tedu_Computational_Statistics/blob/main/6/W6a.ipynb)


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import stats
import pickle

#Monte Carlo Integration on Unbounded Integral (change of variables)
# $\Phi(x) = \int_{-\infty}^x \frac{1}{\sqrt{2 \pi}} e^{-t^2/2} dt$
# Let's use Monte Carlo Integration to find an approximate value of this integral.
# Remember that if $x >0$, we have the following relation:
# $\Phi(-x) = 1- \phi(x)$.
# So if $x>0$, by an obvious change of variables, we can do the following:
# $\Phi(x) = 0.5 + \int_0^1 \frac{x}{\sqrt{2 \pi}} e^{-u^2x^2/2} du$.


In [33]:
def Phi(x, n_samples = 1000):
  sample = np.random.rand(n_samples)
  f = lambda u: x/(np.sqrt(2*np.pi))*np.exp(-(u*x)**2/2)  
  ### Herer the function f is monotone in u
  return f(sample)

In [27]:
montecarlo_var = np.var(Phi(0.01), ddof = 1)

In [20]:
montecarlo_var

0.054023544846244324

# Lets use Anthitetic Variables Now!!!

In [14]:
def Phi_a(x, n_samples = 1000):
  sample = np.random.rand(n_samples)
  f = lambda u: x/(2*np.sqrt(2*np.pi))*(np.exp(-(u*x)**2/2)+np.exp(-((1-u)*x)**2/2))
  return f(sample)

In [31]:
ant_var = np.var(Phi_a(0.01, 500), ddof = 1)

In [22]:
ant_var

7.714636264102961e-05

 # See how much we gained?

In [32]:
100*(montecarlo_var - ant_var)/montecarlo_var
###Great Success!!!!

93.84817253428737

#Control Variates to Reduce Variance

#Use a control variate function to approximate the value of $\int_{0}^1 e^u du$.

# Importance Sampling
# Consider the following integral
# $ \int_0^1 \frac{e^{-x}}{1+x^2} dx$.
# Choose a proper importance functionce for a possible decrease in the variance.

In [125]:
g = lambda x : np.exp(-x)/(1+x**2)
f = lambda x : np.exp(0.5)/(1+x**2)
L = lambda x : f(x)*g(x)

In [126]:
def cov(n_samples = 1000):
  samp = np.random.rand(n_samples)
  f_samp = f(samp)
  g_samp = g(samp)
  return np.mean((f_samp - np.mean(f_samp))*(g_samp - np.mean(g_samp)))

In [127]:
cov()  #### covariance is positive (we are good)

0.06374901607670383

In [128]:
def c_val(n_samples = 1000):
  samp = np.random.rand(n_samples)
  f_samp = f(samp)
  g_samp = g(samp)
  cov = np.mean((f_samp - np.mean(f_samp))*(g_samp - np.mean(g_samp)))
  return -cov/np.var(f_samp, ddof = 1) ### This is the value of c from the formula


In [129]:
def mean(f, n_sample = 1000):
  sample = np.random.rand(n_sample)
  return np.mean(f(sample))

In [130]:
mu = mean(f)
c = c_val()
print(c)

-0.9007903096842454


In [131]:
def sample(c, mu, n_samples = 1000):
  samp = np.random.rand(n_samples)
  func = lambda x: g(x) - c*(f(x) - mu)
  return func(samp)

In [132]:
np.mean(sample(c, mu)) ### this is control variate version with f chose as above

0.5391106745501651

In [133]:
mean(g) ###this is standard monte carlo integration

0.5116339874111316

# How much we gained?

In [134]:
def var_test(c, mu, n_samples = 1000):
  samp = np.random.rand(n_samples)
  func1 = lambda x: g(x) + c*(f(x) - mu)

  return np.var(func1(samp), ddof = 1), np.var(g(samp), ddof = 1)

In [138]:
var_1, var_2 = var_test(c, mu)

In [139]:
100*(var_2- var_1)/var_2

94.73223960370015